In [1]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

In [2]:
@registry.register_problem
class TRANSLATION32k(translate.TranslateProblem):

    @property
    def additional_training_datasets(self):
        """Allow subclasses to add training datasets."""
        return []

In [3]:
PROBLEM = 'translatio_n32k'
problem = problems.problem(PROBLEM)

In [4]:
import tensorflow as tf
import os

vocab_file = "t2t/data/vocab.translatio_n32k.32768.subwords"
ckpt_path = tf.train.latest_checkpoint(os.path.join('t2t/train-base'))
vocab_file, ckpt_path

('t2t/data/vocab.translatio_n32k.32768.subwords',
 't2t/train-base/model.ckpt-75000')

In [5]:
from t import text_encoder

encoder = text_encoder.SubwordTextEncoder(vocab_file)

In [48]:
from tensor2tensor.layers import modalities
from tensor2tensor.layers import common_layers

def top_p_logits(logits, p):
    with tf.variable_scope('top_p_logits'):
        logits_sort = tf.sort(logits, direction = 'DESCENDING')
        probs_sort = tf.nn.softmax(logits_sort)
        probs_sums = tf.cumsum(probs_sort, axis = 1, exclusive = True)
        logits_masked = tf.where(
            probs_sums < p, logits_sort, tf.ones_like(logits_sort) * 1000
        )  # [batchsize, vocab]
        min_logits = tf.reduce_min(
            logits_masked, axis = 1, keepdims = True
        )  # [batchsize, 1]
        return tf.where(
            logits < min_logits,
            tf.ones_like(logits, dtype = logits.dtype) * -1e10,
            logits,
        )


def sample(translate_model, features):
    logits, losses = translate_model(features)
    logits_shape = common_layers.shape_list(logits)
    logits_p = logits[:,0,:,0,:] / translate_model.hparams.sampling_temp
    logits_p = top_p_logits(logits_p, translate_model.hparams.top_p)
    reshaped_logits = tf.reshape(logits_p, [-1, logits_shape[-1]])
    choices = tf.multinomial(reshaped_logits, 1)
    samples = tf.reshape(choices, logits_shape[:-1])
    return samples, logits, losses

def nucleus_sampling(translate_model, features, decode_length):
    """A slow greedy inference method.
    Quadratic time in decode_length.
    Args:
      features: an map of string to `Tensor`
      decode_length: an integer.  How many additional timesteps to decode.
    Returns:
      A dict of decoding results {
          "outputs": integer `Tensor` of decoded ids of shape
              [batch_size, <= decode_length] if beam_size == 1 or
              [batch_size, top_beams, <= decode_length]
          "scores": None
          "logits": `Tensor` of shape [batch_size, time, 1, 1, vocab_size].
          "losses": a dictionary: {loss-name (string): floating point `Scalar`}
      }
    """
    if not features:
        features = {}
    inputs_old = None
    if 'inputs' in features and len(features['inputs'].shape) < 4:
        inputs_old = features['inputs']
        features['inputs'] = tf.expand_dims(features['inputs'], 2)
    # Save the targets in a var and reassign it after the tf.while loop to avoid
    # having targets being in a 'while' frame. This ensures targets when used
    # in metric functions stays in the same frame as other vars.
    targets_old = features.get('targets', None)

    target_modality = translate_model._problem_hparams.modality['targets']

    def infer_step(recent_output, recent_logits, unused_loss):
        """Inference step."""
        if not tf.executing_eagerly():
            if translate_model._target_modality_is_real:
                dim = translate_model._problem_hparams.vocab_size['targets']
                if dim is not None and hasattr(
                    translate_model._hparams, 'vocab_divisor'
                ):
                    dim += (-dim) % translate_model._hparams.vocab_divisor
                recent_output.set_shape([None, None, None, dim])
            else:
                recent_output.set_shape([None, None, None, 1])
        padded = tf.pad(recent_output, [[0, 0], [0, 1], [0, 0], [0, 0]])
        features['targets'] = padded
        # This is inefficient in that it generates samples at all timesteps,
        # not just the last one, except if target_modality is pointwise.
        samples, logits, losses = sample(translate_model, features)
        # Concatenate the already-generated recent_output with last timestep
        # of the newly-generated samples.
        top = translate_model._hparams.top.get(
            'targets', modalities.get_top(target_modality)
        )
        if getattr(top, 'pointwise', False):
            cur_sample = samples[:, -1, :, :]
        else:
            cur_sample = samples[
                :, common_layers.shape_list(recent_output)[1], :, :
            ]
        if translate_model._target_modality_is_real:
            cur_sample = tf.expand_dims(cur_sample, axis = 1)
            samples = tf.concat([recent_output, cur_sample], axis = 1)
        else:
            cur_sample = tf.to_int64(tf.expand_dims(cur_sample, axis = 1))
            samples = tf.concat([recent_output, cur_sample], axis = 1)
            if not tf.executing_eagerly():
                samples.set_shape([None, None, None, 1])

        # Assuming we have one shard for logits.
        logits = tf.concat([recent_logits, logits[:, -1:]], 1)
        loss = sum([l for l in losses.values() if l is not None])
        return samples, logits, loss

    # Create an initial output tensor. This will be passed
    # to the infer_step, which adds one timestep at every iteration.
    if 'partial_targets' in features:
        initial_output = tf.to_int64(features['partial_targets'])
        while len(initial_output.get_shape().as_list()) < 4:
            initial_output = tf.expand_dims(initial_output, 2)
        batch_size = common_layers.shape_list(initial_output)[0]
    else:
        batch_size = common_layers.shape_list(features['inputs'])[0]
        if translate_model._target_modality_is_real:
            dim = translate_model._problem_hparams.vocab_size['targets']
            if dim is not None and hasattr(
                translate_model._hparams, 'vocab_divisor'
            ):
                dim += (-dim) % translate_model._hparams.vocab_divisor
            initial_output = tf.zeros(
                (batch_size, 0, 1, dim), dtype = tf.float32
            )
        else:
            initial_output = tf.zeros((batch_size, 0, 1, 1), dtype = tf.int64)
    # Hack: foldl complains when the output shape is less specified than the
    # input shape, so we confuse it about the input shape.
    initial_output = tf.slice(
        initial_output, [0, 0, 0, 0], common_layers.shape_list(initial_output)
    )
    target_modality = translate_model._problem_hparams.modality['targets']
    if (
        target_modality == modalities.ModalityType.CLASS_LABEL
        or translate_model._problem_hparams.get('regression_targets')
    ):
        decode_length = 1
    else:
        if 'partial_targets' in features:
            prefix_length = common_layers.shape_list(
                features['partial_targets']
            )[1]
        else:
            prefix_length = common_layers.shape_list(features['inputs'])[1]
        decode_length = prefix_length + decode_length

    # Initial values of result, logits and loss.
    result = initial_output
    vocab_size = translate_model._problem_hparams.vocab_size['targets']
    if vocab_size is not None and hasattr(
        translate_model._hparams, 'vocab_divisor'
    ):
        vocab_size += (-vocab_size) % translate_model._hparams.vocab_divisor
    if translate_model._target_modality_is_real:
        logits = tf.zeros((batch_size, 0, 1, vocab_size))
        logits_shape_inv = [None, None, None, None]
    else:
        # tensor of shape [batch_size, time, 1, 1, vocab_size]
        logits = tf.zeros((batch_size, 0, 1, 1, vocab_size))
        logits_shape_inv = [None, None, None, None, None]
    if not tf.executing_eagerly():
        logits.set_shape(logits_shape_inv)

    loss = 0.0

    def while_exit_cond(
        result, logits, loss
    ):  # pylint: disable=unused-argument
        """Exit the loop either if reach decode_length or EOS."""
        length = common_layers.shape_list(result)[1]

        not_overflow = length < decode_length

        if translate_model._problem_hparams.stop_at_eos:

            def fn_not_eos():
                return tf.not_equal(  # Check if the last predicted element is a EOS
                    tf.squeeze(result[:, -1, :, :]), text_encoder.EOS_ID
                )

            not_eos = tf.cond(
                # We only check for early stopping if there is at least 1 element (
                # otherwise not_eos will crash).
                tf.not_equal(length, 0),
                fn_not_eos,
                lambda: True,
            )

            return tf.cond(
                tf.equal(batch_size, 1),
                # If batch_size == 1, we check EOS for early stopping.
                lambda: tf.logical_and(not_overflow, not_eos),
                # Else, just wait for max length
                lambda: not_overflow,
            )
        return not_overflow

    result, logits, loss = tf.while_loop(
        while_exit_cond,
        infer_step,
        [result, logits, loss],
        shape_invariants = [
            tf.TensorShape([None, None, None, None]),
            tf.TensorShape(logits_shape_inv),
            tf.TensorShape([]),
        ],
        back_prop = False,
        parallel_iterations = 1,
    )
    if inputs_old is not None:  # Restore to not confuse Estimator.
        features['inputs'] = inputs_old
    # Reassign targets back to the previous value.
    if targets_old is not None:
        features['targets'] = targets_old
    losses = {'training': loss}
    if 'partial_targets' in features:
        partial_target_length = common_layers.shape_list(
            features['partial_targets']
        )[1]
        result = tf.slice(
            result, [0, partial_target_length, 0, 0], [-1, -1, -1, -1]
        )
    return {
        'outputs': result,
        'scores': None,
        'logits': logits,
        'losses': losses,
    }

In [65]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.top_p = tf.placeholder(tf.float32, None)
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        self.features = features
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        self.translate_model = translate_model
        logits, _ = translate_model(features)
        translate_model.hparams.top_p = self.top_p
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=5, 
                top_beams=1, alpha=1.0)["outputs"]
            self.nucleus_result = nucleus_sampling(translate_model, features, maxlen_decode)["outputs"]
            self.nucleus_result = self.nucleus_result[:,:,0,0]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')
        self.nucleus_result = tf.identity(self.nucleus_result, name = 'nucleus')
        
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.label_smoothing to 0.0


...

INFO:tensorflow:Transforming feature 'targets' with symbol_modality_25385_512.targets_bottom


INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_25385_512.top


INFO:tensorflow:Transforming body output with symbol_modality_25385_512.top


In [66]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from t2t/train-base/model.ckpt-75000


INFO:tensorflow:Restoring parameters from t2t/train-base/model.ckpt-75000


In [68]:
string = 'Religion usually means trust in God, or a supernatural and supernatural power like God, as well as practices and institutions associated with that belief. Religion and belief are two very relevant things. But Religion has a broader meaning, which refers to a system of cohesive belief, and this belief is about the divine aspect'
encoded = encoder.encode(string) + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], feed_dict = {model.X: [encoded], model.top_p: 0.7})
encoder.decode(f[0]), encoder.decode(b[0]), encoder.decode(n[0])

('Agama biasanya bermaksud kepercayaan kepada Tuhan, atau kekuatan ghaib dan ghaib seperti Tuhan, serta amalan dan institusi yang berkaitan dengan kepercayaan itu. Agama dan kepercayaan adalah dua perkara yang sangat relevan. Tetapi Agama mempunyai makna yang lebih luas, yang merujuk kepada sistem kepercayaan yang kohesif, dan kepercayaan ini adalah mengenai aspek ilahi<EOS>',
 'Agama biasanya bermaksud kepercayaan kepada Tuhan, atau kekuatan ghaib dan ghaib seperti Tuhan, serta amalan dan institusi yang berkaitan dengan kepercayaan itu. Agama dan kepercayaan adalah dua perkara yang sangat relevan. Tetapi Agama mempunyai makna yang lebih luas, yang merujuk kepada sistem kepercayaan kohesif, dan kepercayaan ini adalah mengenai aspek ilahi<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 'Agama biasanya bermaksud kepercayaan kepada Tuhan, atau kekuatan ghaib dan NarcPeter seperti iblis, serta amalan dan institusi yang berkaitan dengan kepercayaan itu. Ag

In [70]:
string = 'Articulate (Well-spoken, Expressive) <> Hmmm yes Horacio Calcaterra is a sportsman that currently is playing for Sporting Cristal which is located in Torneo Descentralizado. <> He currently plays for Sporting Cristal in the Torneo Descentralizado.'
encoded = encoder.encode(string) + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], feed_dict = {model.X: [encoded], model.top_p: 0.7})
encoder.decode(f[0]), encoder.decode(b[0]), encoder.decode(n[0])

('Articulate (Bahasa yang diucapkan dengan baik, Ekspresif) <> Hmmm ya Horacio Calcaterra adalah seorang ahli sukan yang kini bermain untuk Sporting Cristal yang terletak di Torneo Descentralizado. <> Dia kini bermain untuk Sporting Cristal di Torneo Descentralizado.<EOS>',
 'Artikulat (dituturkan dengan baik, Ekspresif) <> Hmmm ya Horacio Calcaterra adalah seorang ahli sukan yang kini bermain untuk Sporting Cristal yang terletak di Torneo Descentralizado. <> Dia kini bermain untuk Sporting Cristal di Torneo Descentralizado.<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 'Artikulasi (dituturkan dengan baik, Ekspresif) <> Hmmm ya Horacio Calcaterra adalah ahli sukan yang kini bermain untuk Sporting Cristal yang terletak di Torneo Descentralizado. <> Dia Volcano bermain offers Tugu preservhalasacred par tugas di Torneo Descentralizado.<EOS>')

In [73]:
string = ('TANGKAK - Tan Sri Muhyiddin Yassin said he did not want to touch on '
 'political issues at the moment, instead focusing on the welfare of the '
 "people and efforts to revitalize the affected country's economy following "
 'the Covid-19 pandemic. The prime minister explained the matter when speaking '
 'at a Leadership Meeting with Gambir State Assembly (DUN) leaders at the '
 'Bukit Gambir Multipurpose Hall today.')
encoded = encoder.encode(string) + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], feed_dict = {model.X: [encoded], model.top_p: 0.3})
encoder.decode(f[0]), encoder.decode(b[0]), encoder.decode(n[0])

('TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh isu politik pada masa ini, sebaliknya memberi tumpuan kepada kebajikan rakyat dan usaha untuk memulihkan ekonomi negara yang terjejas susulan pandemik Covid-19 Beliau berkata demikian ketika berucap pada Majlis Perjumpaan Bersama Pemimpin Dewan Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.<EOS>',
 'TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh isu politik pada masa ini, sebaliknya memberi tumpuan kepada kebajikan rakyat dan usaha untuk memulihkan ekonomi negara yang terjejas susulan pandemik Covid-19 Perdana Menteri menjelaskan perkara itu ketika berucap pada Majlis Perjumpaan Bersama Pemimpin Dewan Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [ ]:
from unidecode import unidecode

In [ ]:
string = "Partner's persona: i am in medical school. Partner's persona: i really wanted to be an actor. your persona: i've 2 kids. your persona: i love flowers."
string = unidecode(string)
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

In [ ]:
string = 'Emmerdale is the debut studio album,songs were not released in the U.S. .[EENND] These songs were not released in the U.S. edition of said album and were previously unavailable on any U.S. release.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])